# Sincronización

La `sincronización` es un concepto en las ciencias de la computación que especifica ciertos tipo de mecanismo para asegurar que no más de una tarea (proceso/hilo) concurrente pueda procesar o ejectuar una porción del programa en un instante de tiempo. Esta porción de código se conoce como **sección crítica**. 

Cuando en un programa una tarea accede a una sección crítica las demás tareas tienen que esperar hasta finalizar la tarea que está en ejecución. El objetivo de la sincronización es evitar cualquier corrupción en el manejo de recursos compartidos, si solo una tarea ejecuta la sección crítica del programa en una única unidad de tiempo se garantiza que se evitará la aparición de conflictos provocados por las múltiples tareas producto de la concurrencia.

Los hilos o *threads* son procesos ligeros que se ejecutan en el mismo espacio de memoria, por tanto complica el uso de variables dado que múltiples hilos pueden escribir en la *misma localidad de memoria*, lo que no sucede con los procesos que deben utilizar un mecanismo de comunicación para compartir datos, aunque esto **no asegura** la sincronización.

Por otra parte, el intérprete de Python proporciona un mecanismo de seguridad para los hilos (_thread-safe mechanism_): **GIL** (Global intepreter Lock), el **interprete global de bloqueo**, este mecanismo previene de conflictos entre hilos al ejecutar una sola instrucción al mismo tiempo. No obstante el GIL, la sincronización **no está asegurada**.

## Ejemplos de sincronización

### Secuencial

In [601]:
import time

balance = 200
trans = 100000

def deposit():
    global balance
    balance = balance + 1
    
def withdraw():
    global balance
    balance = balance - 1

t = time.time()     
for tr in range(trans):
    deposit()
    withdraw()
    
print(balance)
print(time.time()- t)

200
0.03161811828613281


### Procesos

In [569]:
import time
import multiprocessing

def deposit(balance):
    for i in range(10000):
        # sección crítica
        balance.value = balance.value + 1
        
def withdraw(balance):
    for i in range(10000):
        # sección crítica
        balance.value = balance.value - 1

# variable compartida     
balance = multiprocessing.Value('i', 200)

# creación de procesos 
d = multiprocessing.Process(target=deposit, args=(balance,))
w = multiprocessing.Process(target=withdraw, args=(balance,))
d.start()
w.start()
d.join()
w.join()
print(balance.value)

544


## Hilos:

In [612]:
import time
import threading

# variable compartida
balance = 200

def deposit():
    global balance
    for i in range(100000):
        # sección crítica
        balance = balance + 1
    print("B1", balance)
        
def withdraw():
    global balance
    for i in range(100000):
        # sección crítica
        balance = balance - 1    
    print("B2", balance)

t = time.time()
# creación de hilos       
d = threading.Thread(target=deposit, args=())
w = threading.Thread(target=withdraw, args=())

d.start()
w.start()
d.join()
w.join()

print(time.time() - t)
print(balance)

B1 B2 200
100200
0.03026747703552246
200


### Dificultades con los hilos
+ Los hilos no tienen un comportamiento determinista
+ La planificación la realiza el sistema operativo, no el intérprete de Python
+ Es impredecible la ejecución de un hilo, por lo tanto, el código debe ser seguro para cada hilo (*thread safe*)

## Bloqueo

Un mecanismo básico de sincronización es el **bloqueo** de recursos compartidos, el propósito de este bloqueo es proteger un recurso particular del acceso múltiple por diversas tareas. Entre los recursos compartidos se consideran: memoria, archivos, bases de datos, etc.

En Python este bloqueo se implementa por medio de los métodos siguientes:
    
`Lock`: Bloqueo predeterminado

`acquire()`: Inicia el bloqueo

`release()`: Libera del bloqueo


## Ejemplos de Bloqueo

### Procesos

In [583]:
import time
import multiprocessing

def deposit(balance, lock):
    for i in range(100000):
        lock.acquire()   # inicia bloqueo
        balance.value = balance.value + 1
        lock.release()   # libera del bloqueo
        
def withdraw(balance, lock):
    for i in range(100000):
        lock.acquire()  # inicia bloqueo
        balance.value = balance.value - 1
        lock.release()  # libera del bloqueo

# variable compartida    
balance = multiprocessing.Value('i', 200)
# crea un candado
lock = multiprocessing.Lock()

# crea proceos
d = multiprocessing.Process(target=deposit, args=(balance, lock))
w = multiprocessing.Process(target=withdraw, args=(balance, lock))
d.start()
w.start()
d.join()
w.join()

print(balance.value)

200


### Hilos

In [616]:
# importa time y threading
import time
import threading

# variable compartida
balance = 200
# crea un candado
lock = threading.Lock()

# define función de incremento
def deposit():
    global balance
    for i in range(100000):
        lock.acquire()
        balance = balance + 1
        lock.release()
    print("B1", balance)

    
# define función de incremento
def withdraw():
    global balance
    for i in range(100000):
        lock.acquire()  
        balance = balance - 1
        lock.release()
    print("B2", balance)

# inicia los hilos
t = time.time()
d = threading.Thread(target=deposit, args=())
w = threading.Thread(target=withdraw, args=())
d.start()
w.start()
d.join()
w.join()
print(time.time()-t)
print(balance)

B1 27016
B2 200
0.37801694869995117
200


In [252]:
# Importing the threading module
import threading 
deposit = 100
# Function to add profit to the deposit
def add_profit(): 
    global deposit
    for i in range(100000):
        deposit = deposit + 10
# Function to deduct money from the deposit
def pay_bill(): 
    global deposit
    for i in range(100000):
        deposit = deposit - 10
# Creating threads
thread1 = threading.Thread(target = add_profit, args = ())
thread2 = threading.Thread(target = pay_bill, args = ())
# Starting the threads
thread1.start() 
thread2.start()
# Waiting for both the threads to finish executing 
thread1.join()
thread2.join()
# Displaying the final value of the deposit
print(deposit)

100
